In [34]:
import dash
from dash import dcc, html , Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from stock import transform_data
from candle import candle_figure
import pandas as pd

In [35]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SOLAR], 
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]
                )


In [36]:
current_year = datetime.now().year

In [37]:
month_3 = (datetime.now() - relativedelta(months=3)).strftime('%Y-%m-%d')
month_6 = (datetime.now() - relativedelta(months=6)).strftime('%Y-%m-%d')
year_1 = (datetime.now() - relativedelta(years=1)).strftime('%Y-%m-%d')
year_5 = (datetime.now() - relativedelta(years=5)).strftime('%Y-%m-%d')

this_year = datetime.now().year
this_month = datetime.now().month
this_day = datetime.now().day


In [38]:
df = transform_data("ITUB4.SA", 'max')

In [39]:
br_df = pd.read_html("https://www.dadosdemercado.com.br/bolsa/acoes").copy()
tickers = br_df[0]['Ticker']
tickers = [tickers + ".SA" for tickers in tickers]

In [40]:
main_config = {
    'layout': {
        'template': 'plotly_white',
        'xaxis': {'showgrid': False },
        'legend': { 'orientation': 'h', 'y': 1, 'yanchor': 'bottom', 'x': 0.5, 'xanchor': 'center'}

    }
}

In [41]:
fig = candle_figure(df)

In [42]:
app.layout = dbc.Container([
    dbc.Row([
        html.Header("Stock Market Analytics",className='text-center fs-1 text-primary mb-4'),    
        html.Div([
            html.P(
            [
                html.Span(children="Search and see major statistics",
                            style={'fontWeight': 'bold'}),
                html.Span(id='results', className='text-primary', 
                            style={'fontWeight': 'bold'}),
            ],
            style={'display': 'flex', 'justify-content': 'space-between'}
        ),
        
            ]), 
    ]),
    dbc.Row([
        dcc.Dropdown(tickers, "ITUB4.SA", id='dropdown', className="dropdown" ),
    ]),
    dbc.Row([
        html.Div([
            dbc.ButtonGroup(
                [
                    dbc.Button("3M", id='bb3M',  className="mt-1", n_clicks=0),
                    dbc.Button("6M", id='bb6M',  className="mt-1", n_clicks=0),
                    dbc.Button("1Y", id='bb1Y',  className="mt-1", n_clicks=0),
                    dbc.Button("5Y", id='bb5Y',  className="mt-1", n_clicks=0),
                    dbc.Button("MAX", id='bb_max',  className="mt-1", n_clicks=0)
                ], style={'display': 'flex', 'align-items': 'center'}
            ),
            html.Div([
                dcc.DatePickerSingle(
                    id='date-picker-start',
                    date=datetime(2000, 12, 22) ,
                    display_format='YYYY-MM-DD',
                    min_date_allowed=datetime(2000, 12, 22),
                    max_date_allowed=datetime(this_year, this_month, this_day),
                    style={'margin': '5px 5px', 'align-items': 'center'}
            ),
                dcc.DatePickerSingle(
                    id='date-picker-end',
                    date=datetime(this_year, this_month, this_day) ,
                    display_format='YYYY-MM-DD',
                    min_date_allowed=datetime(2000, 12, 22),
                    max_date_allowed=datetime(this_year, this_month, this_day),
            ), 
            ], style={'display': 'flex', 'align-items': 'center'})
            
        ], style={'display': 'flex', 'justify-content': 'space-between', 'align-items': 'center' }),
            
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id='stock-graph',        
                figure = {
                    'data': [
                        go.Scatter(x=df.index, y=df['Close'],
                                    mode='lines', name="ITUB4.SA",
                                    ),
                    ],
                    'layout': go.Layout(
                        **main_config['layout'],
                        title=f'Stock Market Data - ITUB4.SA',
                        yaxis={'fixedrange': True, 'title': 'Close Values', 'tickformat': ',.2f', 'tickprefix':'R$'},
                    ), 
                },style={'margin': '10px 0'}
            ),
        ]),
        dbc.Col([
            dcc.Graph(
                id='return-graph',        
                figure = {
                    'data': [
                        go.Scatter(x=df.index, y=df['Return_Stock'] * 100, fill='tonexty',
                                    mode='lines', name="Stock Return", ),
                        go.Scatter(x=df.index, y=df['Return_CDI'] * 100,
                                    mode='lines', name="CDI Return"),
                        go.Scatter(x=df.index, y=df['Return_Inflation'] * 100,
                                    mode='lines', name="Inflation Return")
                    ],
                    'layout': go.Layout(
                        **main_config['layout'],
                        title=f'CDI Return vs ITUB4.SA Return',
                        yaxis={'fixedrange': True, 'title': 'Investiments','tickformat': ',.0f', 'ticksuffix':'%'},
                    ), 
                }
                ,style={'margin': '10px 0'}
            ),
        ])
    ]),
    dbc.Row([
        dcc.Graph(
            id='candle-graph',
            
            figure=candle_figure(df)
        )
    ]),
    dbc.Row([
        html.Footer(
            f"Powered by dash - Gustavo Neves {current_year}", className='text-primary', id='test'
        )])
    ])


In [43]:
@app.callback(
    [
        Output('stock-graph', 'figure'),
        Output('return-graph', 'figure'),
        Output('candle-graph', 'figure'),
        Output('results', 'children'),
        Output('date-picker-start', 'date'),
        Output('date-picker-end', 'date')
    ],
    [
        Input('bb3M', 'n_clicks'),
        Input('bb6M', 'n_clicks'),
        Input('bb1Y', 'n_clicks'),
        Input('bb5Y', 'n_clicks'),
        Input('bb_max', 'n_clicks'),
        Input('dropdown', 'value'),
        Input('date-picker-start', 'date'),
        Input('date-picker-end', 'date'),
     ],
     
    
)
def update_graph(n_clicks_3M, n_clicks_6M, n_clicks_1Y, n_clicks_5Y, n_clicks_max, selected_df, start, end):



    ctx = dash.callback_context
    button_id = ctx.triggered_id


    if button_id:
        # Determine which button was clicked
        button_value = button_id.split('.')[0]
        selected_period = "MAX"

        if button_value == 'bb3M':
            selected_period = '3M'
        if button_value == 'bb6M':
            selected_period = '6M'
        elif button_value == 'bb1Y':
            selected_period = '1Y'
        elif button_value == 'bb5Y':
            selected_period = '5Y'
        elif button_value == 'bb_max':
            selected_period = 'MAX'
        
        
        if selected_df:
            df = transform_data(selected_df, 'max')
            year = (int(df.index.to_series().dt.year.iloc[0]))
            month = (int(df.index.to_series().dt.month.iloc[0]))
            day = (int(df.index.to_series().dt.day.iloc[0]))
            start_date = datetime(year, month, day)
            end_date = datetime(this_year, this_month, this_day)
            if selected_period == '3M':
                df = transform_data(selected_df, 'None', month_3)
                year = (int(df.index.to_series().dt.year.iloc[0]))
                month = (int(df.index.to_series().dt.month.iloc[0]))
                day = (int(df.index.to_series().dt.day.iloc[0]))
                start_date = datetime(year, month, day)
                end_date = datetime(this_year, this_month, this_day)
            if selected_period == '6M':
                df = transform_data(selected_df, 'None', month_6)
                year = (int(df.index.to_series().dt.year.iloc[0]))
                month = (int(df.index.to_series().dt.month.iloc[0]))
                day = (int(df.index.to_series().dt.day.iloc[0]))
                start_date = datetime(year, month, day)
                end_date = datetime(this_year, this_month, this_day)
            elif selected_period == '1Y':
                df = transform_data(selected_df, 'None', year_1)
                year = (int(df.index.to_series().dt.year.iloc[0]))
                month = (int(df.index.to_series().dt.month.iloc[0]))
                day = (int(df.index.to_series().dt.day.iloc[0]))
                start_date = datetime(year, month, day)
                end_date = datetime(this_year, this_month, this_day)
            elif selected_period == '5Y':
                df = transform_data(selected_df, 'None', year_5)
                year = (int(df.index.to_series().dt.year.iloc[0]))
                month = (int(df.index.to_series().dt.month.iloc[0]))
                day = (int(df.index.to_series().dt.day.iloc[0]))
                start_date = datetime(year, month, day)
                end_date = datetime(this_year, this_month, this_day)

            elif button_value == 'date-picker-start':
                selected_period = start[:10] + ' | ' + end[:10]
                df = transform_data(selected_df, 'None', start[:10], end[:10])
                start_date = start
                end_date = end

            elif button_value == 'date-picker-end':
                selected_period = start[:10] + ' | ' + end[:10]
                df = transform_data(selected_df, 'None', start[:10], end[:10])
                start_date = start
                end_date = end

            figure1 = {
                'data': [
                    go.Scatter(x=df.index, y=df['Close'],
                                mode='lines', name="ITUB4.SA",
                                ),
                    ],
                'layout': go.Layout(
                    **main_config['layout'],
                    title=f'Stock Market Data - {selected_df}',
                    yaxis={'fixedrange': True, 'title': 'Close Values', 'tickformat': ',.2f', 'tickprefix':'R$'},
                )
            }
            figure2 = {
                'data': [
                    go.Scatter(x=df.index, y=df['Return_Stock'] * 100, fill='tonexty',
                                mode='lines', name="Stock Return" ),
                    go.Scatter(x=df.index, y=df['Return_CDI'] * 100,
                                mode='lines', name="CDI Return"),
                    go.Scatter(x=df.index, y=df['Return_Inflation'] * 100,
                                mode='lines', name="Inflation Return")
                ],
                'layout': go.Layout(
                    **main_config['layout'],
                    title=f'CDI Return vs {selected_df} Return',
                    yaxis={'fixedrange': True, 'title': 'Investiments', 'tickformat': ',.0f', 'ticksuffix':'%'},
                ),
            }
            # reshape the candlestick graph if selected period be greater than 6 months
            if len(df) < 110:
                candle_df = df
            elif len(df) > 110 and len(df) < 247 :
                candle_df = df.resample('W-Mon').first()
            else:
                candle_df = df.resample('MS').first()

            figure3 = candle_figure(candle_df ,selected_df)
            return figure1, figure2, figure3 , f"Selected period: {selected_period}", start_date, end_date

    # Return if no button was selected 
    if selected_df:
        df = transform_data('ITUB4.SA', 'max')
        year = (int(df.index.to_series().dt.year.iloc[0]))
        month = (int(df.index.to_series().dt.month.iloc[0]))
        day = (int(df.index.to_series().dt.day.iloc[0]))
        start_date = datetime(year, month, day)
        end_date = datetime(this_year, this_month, this_day) 

        figure1 = {
                    'data': [
                        go.Scatter(x=df.index, y=df['Close'],
                                    mode='lines', name="ITUB4.SA")
                    ],
                    'layout': go.Layout(
                        **main_config['layout'],
                        title=f'Stock Market Data - ITUB4.SA',
                        yaxis={'fixedrange': True, 'title': 'Close Values', 'tickformat': ',.2f', 'tickprefix':'R$'},
                    )
        }

        figure2 = {
            'data': [
                go.Scatter(x=df.index, y=df['Return_Stock'] * 100, fill='tonexty',
                            mode='lines', name="Stock Return" ),
                go.Scatter(x=df.index, y=df['Return_CDI'] * 100,
                            mode='lines', name="CDI Return"),
                go.Scatter(x=df.index, y=df['Return_Inflation'] * 100,
                            mode='lines', name="Inflation Return")
            ],
            'layout': go.Layout(
                **main_config['layout'],
                title=f'CDI Return vs {selected_df} Return',
                yaxis={'fixedrange': True, 'title': 'Investiments', 'tickformat': ',.0f', 'ticksuffix':'%'},
            ),
        }
        # reshape the candlestick graph if selected period be greater than 6 months
        if len(df) < 110:
            candle_df = df
        elif len(df) > 110 and len(df) < 247 :
            candle_df = df.resample('W-Mon').first()
        else:
            candle_df = df.resample('MS').first()
            
        figure3 = candle_figure(candle_df, selected_df)
        return figure1, figure2, figure3, "Selected period: MAX", start_date, end_date
    return {'data': [], 'layout': {}}, {'data': [], 'layout': {}}, "Selected period: None", None, None


In [ ]:
if __name__ == "__main__":
    app.run_server(port=5050)

[2024-01-07 19:20:42,347] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Gustavo neves\Downloads\Python\venv\Lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Gustavo neves\Downloads\Python\venv\Lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Gustavo neves\Downloads\Python\venv\Lib\site-packages\flask\app.py", line 867, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Gustavo neves\Downloads\Python\venv\Lib\site-packages\flask\app.py", line 852, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Gustavo nev